In [ ]:
import pathlib

# Setup

## Parse trajectory

In [ ]:
from analyzer.io.trajectory_parser import parse_trajectory

trajectory = parse_trajectory(pathlib.Path("demos/crossing/CROSSING_90_D_6.txt"), 16.)

## Define geometry

In [ ]:
from analyzer.data.geometry import Geometry

### Define geometry in code

In [ ]:
import pygeos
geometry = Geometry(pygeos.polygons([(-9, 3), (-2, 3), (-2, 10), (2, 10), (2, 3), (9, 3), (9, -1), (2, -1), (2, -8), (-2, -8), (-2, -1), (-9, -1), (-9, 3)]))

### Define geometry in file

The same format as for JuPedSim geometries can be used as well directly from files.

**Note:** Currently obstacles will not be parsed from the file!

In [ ]:
from analyzer.io.geometry_parser import parse_geometry

geometry = parse_geometry('demos/crossing/geometry.xml')

## Define measurement areas and lines

In [ ]:
import pygeos

measurement_area = pygeos.polygons([(-2, -1), (-2, 3), (2, 3), (2, -1)])

measurement_line_left = pygeos.linestrings([(-3, 3), (-3, -1)])
measurement_line_right = pygeos.linestrings([(3, 3), (3, -1)])

measurement_line_top = pygeos.linestrings([(-2, 4), (2, 4)])
measurement_line_bottom = pygeos.linestrings([(-2, -2), (2, -2)])

## Plot trajectories, geometry, and measurement areas/lines

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(15, 20))
ax1 = fig.add_subplot(111, aspect="equal")

# Plot geometry
ax1.plot(*pygeos.to_shapely(geometry.walkable_area).exterior.xy, color='k')

## Plot measurement area
ax1.plot(*pygeos.to_shapely(measurement_area).exterior.xy, color='g')
ax1.fill(*pygeos.to_shapely(measurement_area).exterior.xy, color='g', alpha=0.3)

## Plot measurement lines
ax1.plot(*pygeos.to_shapely(measurement_line_left).xy, color='b')
ax1.plot(*pygeos.to_shapely(measurement_line_right).xy, color='b')
ax1.plot(*pygeos.to_shapely(measurement_line_top).xy, color='orange')
ax1.plot(*pygeos.to_shapely(measurement_line_bottom).xy, color='orange')

# Plot trajectories
for id, ped in trajectory.data.groupby("ID"):
    p = ax1.plot(ped["X"], ped["Y"], label=id, alpha=0.1, color='r')
    ax1.scatter(
        ped[ped.frame == ped.frame.max()]["X"],
        ped[ped.frame == ped.frame.max()]["Y"],
        c=p[-1].get_color(),
        marker="x",
    )
plt.show()

# Methods A

## Compute n-t and flow

In [ ]:
from analyzer.methods.flow_calculator import compute_n_t
from analyzer.methods.flow_calculator import compute_flow

from analyzer.methods.velocity_calculator import compute_individual_velocity

individual_speed = compute_individual_velocity(trajectory.data, trajectory.frame_rate, 10)

nt_left, crossing_left = compute_n_t(trajectory.data, measurement_line_left, trajectory.frame_rate)
nt_right, crossing_right = compute_n_t(trajectory.data, measurement_line_right, trajectory.frame_rate)

delta_t = 100
flow_left = compute_flow(nt_left, crossing_left, individual_speed, delta_t, trajectory.frame_rate)
flow_right = compute_flow(nt_right, crossing_right, individual_speed, delta_t, trajectory.frame_rate)

## Plot n-t diagram

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(7, 7))
ax1 = fig.add_subplot(111)

ax1.plot(nt_left.index, nt_left['Cumulative pedestrians'], label='left')
ax1.plot(nt_right.index, nt_right['Cumulative pedestrians'], label='right')

ax1.legend()
plt.show()